### Preparation for running Nanopore methylation calling on NCI

This script does the following:

- pull out all aligned fastq for each contig using previously mapped Nanopores reads with Minimap2
- pack them up 
- pull out all corresponding fast5 files and pack them up

In [2]:
import os
import pysam
import pandas as pd
import glob
import tarfile #compress fast5
from Bio import SeqIO

In [3]:
#first we need to define the base dirs
DIRS ={}
#DIRS['BASE'] = '/home/ap/mock_up/methylation_calling/nanopore' #home computer. hash out later
DIRS['BASE'] = '/home/anjuni/methylation_calling/nanopore' #fisher
DIRS['BAM_INPUT'] = os.path.join(DIRS['BASE'], 'input', 'minimap2_alignments' )
DIRS['FAST5_INPUT'] = os.path.join(DIRS['BASE'], 'input', 'all_fast5') #fix for all runs and copy from nci to right directory
DIRS['FASTQ_OUT'] = os.path.join(DIRS['BASE'], 'input', 'split_fastq')
DIRS['FAST5_OUT'] = os.path.join(DIRS['BASE'], 'input', 'split_fast5')
DIRS['REF_OUT'] = os.path.join(DIRS['BASE'], 'input', 'split_ref')

#fix this here for reference
DIRS['REF'] = '/home/anjuni/Pst_104_v13_assembly/' # Pst_104E_v13_ph_ctg.fa

In [48]:
#define functions

#to compress the fast5 reads mapping contig 19 to a tar.gz file
def make_tarfile(output_filename, file_list):
    with tarfile.open(output_filename, "w:gz") as tar:
        count=0
        for file in file_list:
            tar.add(file, arcname=os.path.basename(file))
            count=count+1
        print(count)

In [5]:
##Get headings

seq_sum_albacore_fh = os.path.join(DIRS['FAST5_INPUT'], 'albacore_fastq/Pst79_run1-4_1d_sequencing_summary.txt')
#only read in the first two columns instead of everything (file name and read ID)
seq_sum_df = pd.read_csv(seq_sum_albacore_fh, sep='\t')

#reduce the size of the seq_sum dataframe to only contain the filename and read_id column thats all we need
small_df = seq_sum_df.iloc[:, [0,1]].copy()


/home/anjuni/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (3,4,5,6,7,8,9,10,11,12,13,14,16,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


seq_sum_df.head # check if it shows the file name and read ID only. (shows all heading)

In [6]:
small_df.head() #check format of dataframe

,filename,read_id
0,rsb0001410_20170719_FAH05512_MN21513_mux_scan_...,12756ced-c758-42f7-a359-ba477a20b3a1
1,rsb0001410_20170719_FAH05512_MN21513_mux_scan_...,15749e40-174c-4bd3-9341-8cb37fde491b
2,rsb0001410_20170719_FAH05512_MN21513_mux_scan_...,23703e5f-144e-405b-aba1-b50ce853ecd9
3,rsb0001410_20170719_FAH05512_MN21513_mux_scan_...,24dd4f66-0a77-4a45-8b5f-8beb2d1576b0
4,rsb0001410_20170719_FAH05512_MN21513_mux_scan_...,2f4921ba-e9a3-44da-90d0-53815af5fcd7


In [7]:
#make second dataframe with read_id first and filename second
columnsTitles=["read_id","filename"]
df2=small_df.reindex(columns=columnsTitles)

In [8]:
df2.drop_duplicates(inplace=True)

In [9]:
df2.to_csv(path_or_buf=os.path.join(DIRS['FASTQ_OUT'], 'read_id_to_fast5'), sep='\t', header=False, index=False)

In [10]:
df2.shape

(1099284, 2)

In [ ]:
# not using this because usign original fastq
!python /home/anjuni/scripts/Basecalling-comparison/fix_read_names.py {os.path.join(DIRS['FASTQ_OUT'], 'pcontig_019_aln.fastq')} {os.path.join(DIRS['FASTQ_OUT'], 'read_id_to_fast5')} | gzip > {os.path.join(DIRS['FASTQ_OUT'], 'output_reads.fastq.gz')}

In [12]:
#quick chech if directories exist
for value in DIRS.values():
    if not os.path.exists(value):
        print('%s does not exist' % value)

In [13]:
##Make file paths for BAM files and reference genome

#we generated the BAM file handle (file path)
bam_pass_fh = os.path.join(DIRS['BAM_INPUT'], 'Pst79_run1-4_1d_pass.minimap2.out.bam')
bam_fail_fh = os.path.join(DIRS['BAM_INPUT'], 'Pst79_run1-4_1d_fail.minimap2.out.bam')

#make a list of paths for BAM files to get mapped reads (for all 4 runs)
bam_fh_list = [os.path.join(DIRS['BAM_INPUT'], x) for x in os.listdir(DIRS['BAM_INPUT']) if x.endswith('.bam')]

######fix this here for reference. use the same as for mapping the long reads
reference_fh = os.path.join(DIRS['REF'], 'Pst_104E_v13_ph_ctg.fa')

split_reference_fh = os.path.join(DIRS['REF_OUT'], 'ref_pcontig_019.fasta') #output folder

In [14]:
#print(bam_fh_list) #check file paths

In [15]:
#just an example to loop over content of a folder
for x in os.listdir(DIRS['BAM_INPUT']):
    if x.endswith('.bam') or x.endswith('.bai'):
        print(x)

Pst79_run1-4_1d_fail.minimap2.out.bam
Pst79_run1-4_1d_pass.minimap2.out.bam
Pst79_run1-4_1d_pass.minimap2.out.bam.bai
Pst79_run1-4_1d_fail.minimap2.out.bam.bai


In [16]:
#print(bam_fh_list) #check for all BAM files
#print(split_reference_fh) #check path

In [17]:
#now we 'read' in an bam file. Really we generated an AlignmentFile object 
bam_pass_file = pysam.AlignmentFile(bam_pass_fh, "rb")
bam_fail_file = pysam.AlignmentFile(bam_fail_fh, "rb")

In [18]:
bam_file_list = [bam_pass_file, bam_fail_file]

In [19]:
bam_file_list

In [20]:
#write a fasta file with all contig_19 reads in reference genome
for seq in  SeqIO.parse(reference_fh, 'fasta'):
    if seq.id == 'pcontig_019':
        SeqIO.write(seq, split_reference_fh, 'fasta')

In [21]:
#get all the reads (not fastq) for contig_19 from the BAM file
contig_19_reads_in_bam = []

count_fail = 0
count_pass = 0

for index,bam in enumerate(bam_file_list):
    for read in bam.fetch(contig='pcontig_019'):
        if index==0:
            count_pass = count_pass + 1
        else:
            count_fail = count_fail + 1
        contig_19_reads_in_bam.append(read) 

In [22]:
len(contig_19_reads_in_bam)

8030

In [38]:
contig_19_reads_in_bam[1]

In [23]:
print('This is the number of fail reads mapped:%i\nThis is the number of pass reads mapped:%i' % (count_fail, count_pass))

This is the number of fail reads mapped:2408
This is the number of pass reads mapped:5622


In [24]:
#this provides the name for all references in the bam file
#bam_file.references

In [39]:
fastq_out_fh = os.path.join(DIRS['FASTQ_OUT'], 'pcontig_019_aln.fastq')

In [26]:
#the outfile for the ids of fastqs mapping to pcontig_019
read_id_fh = os.path.join(DIRS['FASTQ_OUT'], 'pcontig_019_aln.read_id')

#we generate an new file and write out all the aligned reads in fastq format
#we added in an save guard to save out each read only once as it appears that pysam provides some reads in duplicate.
saved_reads = []
with open(read_id_fh, mode='w') as out:
    for read in contig_19_reads_in_bam:
        if read.query_name not in saved_reads:
            print(read.query_name, file=out)
            saved_reads.append(read.query_name)

In [27]:
albacore_fastq_fh = os.path.join('/home/anjuni/methylation_calling/nanopore/input/all_fastq/albacore_fastq', 'Pst79_run1-4_1d_all.fastq')

In [28]:
bbmap_DIR = '/home/anjuni/scripts/BBMap/sh'

In [29]:
#Matching fastq to fast5 note: dingle-threaded, low memory
!filterbyname.sh in={albacore_fastq_fh} out={fastq_out_fh} names={read_id_fh} include=t

java -ea -Xmx392731m -cp /home/anjuni/anaconda3/opt/bbmap-37.95/current/ driver.FilterReadsByName in=/home/anjuni/methylation_calling/nanopore/input/all_fastq/albacore_fastq/Pst79_run1-4_1d_all.fastq out=/home/anjuni/methylation_calling/nanopore/input/split_fastq/pcontig_019_aln.fastq names=/home/anjuni/methylation_calling/nanopore/input/split_fastq/pcontig_019_aln.read_id include=t
Executing driver.FilterReadsByName [in=/home/anjuni/methylation_calling/nanopore/input/all_fastq/albacore_fastq/Pst79_run1-4_1d_all.fastq, out=/home/anjuni/methylation_calling/nanopore/input/split_fastq/pcontig_019_aln.fastq, names=/home/anjuni/methylation_calling/nanopore/input/split_fastq/pcontig_019_aln.read_id, include=t]

Exception in thread "main" java.lang.AssertionError: File /home/anjuni/methylation_calling/nanopore/input/split_fastq/pcontig_019_aln.fastq exists and overwrite=false
	at shared.Tools.testOutputFiles(Tools.java:792)
	at driver.FilterReadsByName.<init>(FilterReadsByName.java:226)
	at d

In [30]:
len(saved_reads)

7167

In [33]:
#we briefly check the longest aligned read
max_length = 0
with pysam.FastxFile(fastq_out_fh) as fh:
    for entry in fh:
        if len(entry.sequence) > max_length:
            max_length = len(entry.sequence)
print('Longest alinged read to contig_19 is %i long'  % max_length)

Longest alinged read to contig_19 is 0 long


In [34]:
### Making fast5 files with the same filenames as fastq, for methylation-calling

#get all the fast5 filenames for the reads that map to contig 19
fast5_names_contig_19 = list(small_df[small_df.read_id.isin(saved_reads)]['filename'])

#this looks for fast5s recursively in all the Fast5_input folder
all_fast5s = [fn for fn in glob.iglob('%s/**/*.fast5' % DIRS['FAST5_INPUT'], recursive=True)]

#this gets the whole path of the fast5s that map to contig_19
fast5s_contig_19_fh = [x for x in all_fast5s if x.split('/')[-1] in fast5_names_contig_19]

In [35]:
#fast5_names_contig_19[0] #check first element

In [47]:
fast5s_contig_19_fh[7166]

'/home/anjuni/methylation_calling/nanopore/input/all_fast5/raw_data/Pst79_4_zipped/20170826_0916_Pst79_RSB_OpenDay_Run_3/fast5/0/rsb_u5580079_20170826_FAH05280_MN16968_mux_scan_Pst79_RSB_OpenDay_Run_3_96315_read_129_ch_94_strand.fast5'

In [37]:
len(saved_reads)

7167

In [44]:
len(set(fast5_names_contig_19))

{'rsb_u5580079_20170826_FAH05280_MN16968_sequencing_run_Pst79_RSB_OpenDay_Run_2_68337_read_1031_ch_205_strand.fast5',
 'rsb0001410_20170809_FAH18496_MN21513_sequencing_run_Ram_Pst79_10K_Run3_94849_read_183_ch_483_strand.fast5',
 'rsb_u5580079_20170826_FAH05280_MN16968_sequencing_run_Pst79_RSB_OpenDay_Run_2_68337_read_8594_ch_445_strand.fast5',
 'rsb_u5580079_20170826_FAH05280_MN16968_sequencing_run_Pst79_RSB_OpenDay_Run_3_48479_read_16897_ch_103_strand.fast5',
 'rsb0001410_20170720__MN21513_mux_scan_Pst79_1_10K_3_25026_read_142_ch_378_strand.fast5',
 'rsb0001410_20170810_FAH18496_MN21513_sequencing_run_Ram_Pst79_10K_Run3_4_30015_read_2566_ch_390_strand.fast5',
 'rsb_u5580079_20170826_FAH05280_MN16968_sequencing_run_Pst79_RSB_OpenDay_Run_2_68337_read_1067_ch_408_strand.fast5',
 'rsb_u5580079_20170826_FAH05280_MN16968_sequencing_run_Pst79_RSB_OpenDay_Run_2_68337_read_121_ch_91_strand.fast5',
 'rsb_u5580079_20170826_FAH05280_MN16968_sequencing_run_Pst79_RSB_OpenDay_Run_2_68337_read_1112_c

#make iteration with all mapped fast5

fast5_mapped=[]
for x in all_fast5s:
    fast5_mapped.append(os.path.join(DIRS["FAST5_OUT"],'pcontig_019_aln_fast5.tar.gz'))

In [49]:
### tarzip the mapped fast5 and move to outfolder
fast5_mapped = os.path.join(DIRS["FAST5_OUT"],'pcontig_019_aln_fast5.tar.gz')
make_tarfile(fast5_mapped, fast5s_contig_19_fh)

7167


#checks

#format of fastq BAM files mapped to contig_19
read_1 = contig_19_reads[0]

#seq_sum_df.columns # to find name column

#show the last line (???)
seq_sum_df[seq_sum_df.read_id == read_1.query_name]['filename'].to_string().split(' ')[-1]


test_fastq = pysam.FastxFile(fastq_out_fh) #BAM file fastq mapping to contig_19
test_fastq.close()

fastq_out.write(read_1.header) # (???)


#Using methods to show fastq lines for fast5
#read_1.header #header (???)

#read_1.qname #deprecated template name (line 1)
#read_1.query_name #template name (line 1)

#read_1.seq #deprecated read fastq seq
#read_1.query_sequence #read fastq sequence
#read_1.query #aligned sequence (line 2)

#read_1.qual #quality score
#read_1.qqual #alignment quality score (line 4)

len(read_1.query) == len(read_1.qqual) #seq and quality score are the same length